# Reinforcement Learning

In [10]:
from environment import Game
import numpy as np

g = Game(4, 4)
q = np.zeros((16, 4))
print(g)
print("Correspondance ID/position:")
print(g.affiche(debug=True))
print("Position du joueur (x) :", g.position)

xo..
....
.¤..
.@..

Correspondance ID/position:
3 7 11 15 
2 6 10 14 
1 5 9 13 
0 4 8 12 

Position du joueur (x) : (3, 0)


## Entrainement

En utilisant le Q-learning, on va entrainer un agent à jouer.

Je défini `g.wrong_action_p` à sa valeur par défaut. Cette variable correspond aux situations aléatoire pouvant survenir dans la vie, traduit dans ce jeu comme un pourcentage de change de faire un mouvement différent de celui demandé.

`alpha` est le taux d'apprentissage, `gamma` est le facteur de réduction de la récompense future.

On réitère une partie sur le même plateau `iterations` fois.

En début de renforcement, on privilégie l'exploration, puis on exploite ce que l'on connait de plus en plus, au fil des parties. 
Ceci est traduit par la variable `epsilon` (tableau de flottant aléatoire) qui est ajouté à la valeur de Q pour la position actuelle.

J'utilise `np.random.randn` qui génère des nombres aléatoires suivant une loi normale gaussienne centrée réduite.
Ou comme le dit si bien la documentation de numpy :
> with random floats sampled from a univariate “normal” (Gaussian) distribution of mean 0 and variance 1

In [18]:
g.wrong_action_p = 0.1
iterations = 1000
alpha = 0.1 # taux d’apprentissage
gamma = 0.9 # interet des récompenses futurs
for _ in range(iterations):
    g.reset()
    is_final = False
    while not is_final:
        state_id = g._get_state()
        # Au début du renforcement, on explore aléatoirement
        # plus on avance, plus on exploite ce que l'on a appris
        epsilon = np.random.randn(1, 4) * (1. / (iterations + 1))
        mov = np.argmax(q[state_id, :] + epsilon)
        (new_state_id, reward, is_final, _) = g.move(mov)
        q[state_id, mov] += alpha * (reward + gamma * np.max(q[new_state_id, :]) - q[state_id, mov])


## Démonstration

In [24]:

# Show a game
g.wrong_action_p = 0.0
g.reset()
is_final = False
score = 0
s = g.__str__()
while not is_final:
    state_id = g._get_state()
    mov = np.argmax(q[state_id, :])
    (new_state_id, reward, is_final, _) = g.move(mov)
    score += reward
    s = '\n'.join([x + '  ' + y for x, y in zip(s.split('\n'), g.__str__().split('\n'))])
print("Score: ", score)
print(s)
print(q)




Score:  7
xo..  .o..  .o..  .o..  .o..
....  x...  ....  ....  ....
.¤..  .¤..  x¤..  .¤..  .¤..
.@..  .@..  .@..  x@..  .x..
        
[[-0.109      -0.1        -0.1         9.68080202]
 [-0.199      -0.199       6.88233248 -0.199     ]
 [-0.58935714 -0.58519851  4.94730056 -0.59855283]
 [-2.18128769 -1.82093062  2.39584532 -1.9       ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-1.          1.10715071 -0.77255306 -0.80190512]
 [ 0.          0.          0.          0.        ]
 [-0.1         2.0821353   0.         -0.1       ]
 [-0.31522042 -0.29701    -0.385219   -0.25632527]
 [-0.55485657 -0.5516245  -0.46619566 -0.458848  ]
 [-1.09       -1.         -0.51789433 -0.5550534 ]
 [-0.199      -0.0466842  -0.1        -0.199     ]
 [-0.31311361 -0.29701    -0.361      -0.3058309 ]
 [-0.46896652 -0.41511377 -0.47334781 -0.3940399 ]
 [-0.4900995  -0.4261888  -0.4988919  -0.4900995 ]]


## Chemin optimal

In [25]:
current = g.start
arrows = ["↑", "←", "↓" , "→"]
while current != g.end:
    current_id = g._position_to_id(*current)
    mov = np.argmax(q[current_id, :])
    print(arrows[mov], end=' ')
    current = (current[0] + g.MOVEMENTS[mov][0], current[1] + g.MOVEMENTS[mov][1])
    

↓ ↓ ↓ → 